# Texas 2k Case

In [1]:
import numpy as np
import pandas as pd

import ams

In [2]:
!ams misc --version

Python   3.12.0
ams      1.0.12
andes    1.9.3
numpy    2.3.0
cvxpy    1.6.0
solvers  CLARABEL, ECOS, ECOS_BB,
         GUROBI, HIGHS, MOSEK, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [3]:
%matplotlib inline

In [4]:
ams.config_logger(stream_level=20)

First, we can read the MATPOWER format case file into a Python dictionary.

In [5]:
mpc = ams.io.matpower.m2mpc("./Texas2k_series24_case3_2024summerpeak.m")

 Case saved by PowerWorld Simulator, version 23, build date October 25, 2023


In [6]:
ngen = mpc["gencost"].shape[0]

This case gives a piecewise lienar cost function with two pairs of points.

For simplification, it is manually mapped to a quadratic cost function with linear term and constant term.

In [7]:
gencost2 = np.concatenate((mpc['gencost'][:, 0:4], np.zeros((ngen, 4))), axis=1)

gencost2[:, 0] = 2  # type of cost function
gencost2[:, 3] = 3  # number of coefficients
gencost2[:, 4] = 0.0  # quadratic coefficient
gencost2[:, 5] = mpc["gencost"][:, 7]  # linear coefficient
gencost2[:, 6] = mpc["gencost"][:, 5]  # constant term

mpc2 = mpc.copy()
mpc2['gencost'] = gencost2

In [8]:
sp2 = ams.system.System()
ams.io.matpower.mpc2system(mpc2, sp2)

sp2.setup()

Zero Line parameters detected, adjusted to default values: rate_b, rate_c, amax, amin.
System set up in 0.0088 seconds.


True

In [9]:
sp3 = ams.load("./Texas2k_series24_case3_2024summerpeak_0_33.RAW",
               setup=False,
               no_output=False,)

Parsing input file "./Texas2k_series24_case3_2024summerpeak_0_33.RAW"...
Input file parsed in 0.5451 seconds.


In [10]:
stg_mp = set(sp2.StaticGen.get_all_idxes())
stg_raw = set(sp3.StaticGen.get_all_idxes())
stg_equal = stg_mp == stg_raw
stg_equal

True

From the inspectin we can see, the two cases have idential generators, so we can add generator costs to the PSSE read case.

In [11]:
for index, row in sp2.GCost.cache.df_in.iterrows():
    sp3.add(model='GCost', **row)

In [12]:
sp3.setup()

Zero Line parameters detected, adjusted to default values: rate_b, rate_c.
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0115 seconds.


True

We can scale down the total load for better convergence if necessary.

In [13]:
p0 = sp3.StaticLoad.get(src='p0', attr='vin',
                        idx=sp3.StaticLoad.get_all_idxes())
sp3.StaticLoad.alter(src='p0', attr='vin',
                     idx=sp3.StaticLoad.get_all_idxes(),
                     value=0.9 * p0)

True

In [14]:
sp3.DCOPF.run(solver='PIQP', ignore_dpp=True)
sp3.DCOPF.export_csv()

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.5389 seconds.
<DCOPF> solved as optimal in 0.0333 seconds, converged in 13 iterations with PIQP.
Report saved to "Texas2k_series24_case3_2024summerpeak_0_33_out.txt" in 0.3987 seconds.


'Texas2k_series24_case3_2024summerpeak_0_33_DCOPF.csv'

You can export the converted cases into either .xlsx or .json for future use.

In [15]:
# ams.io.xlsx.write(sp3, 'Texas2k.xlsx', overwrite=True)
# ams.io.json.write(sp3, 'Texas2k.json', overwrite=True)

Below code can be used to run ACOPF.
Uncomment them to use.

In [16]:
# sp3.ACOPF.run()
# sp3.ACOPF.export_csv()

In [17]:
sp3.DCOPF.pmaxe.e_str

'mul(nctrle, pg0) + mul(ctrle, pmax)'

As above we can see, the `pmaxe` is the effective pmax of generators considering both the online status and controllability.

In [18]:
sp3.DCOPF.pmaxe.v.sum()

np.float64(1347.4291)

In [19]:
sp3.DCOPF.pd.v.sum()

np.float64(633.8973510000001)

Then, in the comparasion of `pd` versus `pmaxe`, we can find there are much more surplus generation than the load.

In [20]:
stg = sp3.DCOPF.pg.get_all_idxes()

gen = pd.DataFrame()
gen['idx'] = stg
gen['u'] = sp3.DCOPF.ug.v
gen['ctrl'] = sp3.DCOPF.ctrl.v
gen['pmax'] = sp3.DCOPF.pmax.v
gen['pmin'] = sp3.DCOPF.pmin.v
gen['pg'] = sp3.DCOPF.pg.v
gen['c2'] = sp3.DCOPF.c2.v
gen['c1'] = sp3.DCOPF.c1.v
gen['c0'] = sp3.DCOPF.c0.v

gen.iloc[25:35].round(4)

,idx,u,ctrl,pmax,pmin,pg,c2,c1,c0
25,26,0.0,1.0,1.219,0.3657,0.0000,0.0,0.0,0.000
26,27,1.0,1.0,0.588,0.1764,0.3893,0.0,0.0,0.000
27,28,1.0,1.0,3.024,0.0000,1.9087,0.0,0.0,0.010
28,29,1.0,1.0,15.245,0.0000,10.7248,0.0,0.0,0.010
29,30,0.0,1.0,3.252,0.9756,-0.0000,0.0,108780.0,0.001
30,31,0.0,1.0,3.252,0.9756,-0.0000,0.0,108780.0,0.001
31,32,1.0,1.0,0.894,0.2682,0.2682,0.0,73410.0,0.001
32,33,1.0,1.0,0.894,0.2682,0.2682,0.0,73410.0,0.001
33,34,1.0,1.0,0.894,0.2682,0.2682,0.0,73410.0,0.001
34,35,0.0,1.0,1.788,0.5364,-0.0000,0.0,86820.0,0.001


From the above DataFrame, we can notice there are some "free" generators.

Next, let's find the "free" generators, and alter their `pmax`.

First, let's find which ``GCost`` are "free" (i.e., with zero cost).

In [21]:
gcost_free = sp3.GCost.find_idx(keys=['c2', 'c1'], values=[[0], [0]],
                                allow_all=True)[0]

Second, we can grab the generators that are "free"

In [22]:
gen_free = sp3.GCost.get(src='gen', attr='v', idx=gcost_free)

Last, their total `pmax` can be calculated:

In [23]:
pg_free = sp3.DCOPF.get(src='pg', attr='v', idx=gen_free)

pg_free.sum()

np.float64(426.33835158600755)

Let's scale it down.

In [24]:
pmax0 = sp3.StaticGen.get(src='pmax', attr='v', idx=gen_free)
pmin0 = sp3.StaticGen.get(src='pmin', attr='v', idx=gen_free)
sp3.StaticGen.alter(src='pmax', attr='v', idx=gen_free,
                    value=0.01 * pmax0)
sp3.StaticGen.alter(src='pmin', attr='v', idx=gen_free,
                    value=0.01 * pmin0)

True

In [25]:
sp3.DCOPF.obj.v

np.float64(26911413.548376776)

In [26]:
sp3.DCOPF.update()

Building system matrices
<DCOPF> reinit OModel due to non-parametric change.
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>


True

In [27]:
sp3.DCOPF.run(solver='PIQP', ignore_dpp=True)

<DCOPF> solved as optimal in 0.0639 seconds, converged in 29 iterations with PIQP.
Report saved to "Texas2k_series24_case3_2024summerpeak_0_33_out.txt" in 0.4825 seconds.


True

In [28]:
sp3.DCOPF.obj.v

np.float64(69587595.75665128)

We can see that after the manipulation, the cost incresed a lot.